In [55]:
import psycopg2
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import classification_report, roc_auc_score

import pickle

import funciones_auxiliares_sql as aux

import importlib

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(aux)

<module 'funciones_auxiliares_sql' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\propio\\personal\\G56\\sql\\prueba\\funciones_auxiliares_sql.py'>

In [2]:
conn = psycopg2.connect("dbname=javier_rojas user=postgres password=2719")
cursor = conn.cursor()


# Parte 1

In [7]:
query_train = """
CREATE TABLE train_cupid(
 age INTEGER,
 height DECIMAL,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs DECIMAL,
 pro_cats DECIMAL,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

query_test = """
CREATE TABLE test_cupid(
 age INTEGER,
 height DECIMAL,
 virgo INTEGER,
 taurus INTEGER,
 scorpio INTEGER,
 pisces INTEGER,
 libra INTEGER,
 leo INTEGER,
 gemini INTEGER,
 aries INTEGER,
 aquarius INTEGER,
 cancer INTEGER,
 sagittarius INTEGER,
 asian INTEGER,
 hispaniclatin INTEGER,
 black INTEGER,
 indian INTEGER,
 pacificislander INTEGER,
 nativeamerican INTEGER,
 middleeastern INTEGER,
 colorado INTEGER,
 newyork INTEGER,
 oregon INTEGER,
 arizona INTEGER,
 hawaii INTEGER,
 montana INTEGER,
 wisconsin INTEGER,
 virginia INTEGER,
 spain INTEGER,
 nevada INTEGER,
 illinois INTEGER,
 vietnam INTEGER,
 ireland INTEGER,
 louisiana INTEGER,
 michigan INTEGER,
 texas INTEGER,
 unitedkingdom INTEGER,
 massachusetts INTEGER,
 northcarolina INTEGER,
 idaho INTEGER,
 mississippi INTEGER,
 newjersey INTEGER,
 florida INTEGER,
 minnesota INTEGER,
 georgia INTEGER,
 utah INTEGER,
 washington INTEGER,
 westvirginia INTEGER,
 connecticut INTEGER,
 tennessee INTEGER,
 rhodeisland INTEGER,
 districtofcolumbia INTEGER,
 canada INTEGER,
 missouri INTEGER,
 germany INTEGER,
 pennsylvania INTEGER,
 netherlands INTEGER,
 switzerland INTEGER,
 mexico INTEGER,
 ohio INTEGER,
 agnosticism INTEGER,
 atheism INTEGER,
 catholicism INTEGER,
 buddhism INTEGER,
 judaism INTEGER,
 hinduism INTEGER,
 islam INTEGER,
 pro_dogs DECIMAL,
 pro_cats DECIMAL,
 spanish INTEGER,
 chinese INTEGER,
 french INTEGER,
 german INTEGER,
 single INTEGER,
 seeing_someone INTEGER,
 available INTEGER,
 employed INTEGER,
 income_between_25_50 INTEGER,
 income_between_50_75 INTEGER,
 income_over_75 INTEGER,
 drugs_often INTEGER,
 drugs_sometimes INTEGER,
 drinks_notatall INTEGER,
 drinks_often INTEGER,
 drinks_rarely INTEGER,
 drinks_socially INTEGER,
 drinks_veryoften INTEGER,
 orientation_gay INTEGER,
 orientation_straight INTEGER,
 sex_m INTEGER,
 smokes_sometimes INTEGER,
 smokes_tryingtoquit INTEGER,
 smokes_whendrinking INTEGER,
 smokes_yes INTEGER,
 body_type_overweight INTEGER,
 body_type_regular INTEGER,
 education_high_school INTEGER,
 education_undergrad_university INTEGER
);
"""

cursor.execute(query_train)
cursor.execute(query_test)

conn.commit()


In [8]:
aux.insertar_csv_a_tabla_postgres("input/train_cupid.csv", "train_cupid", cursor=cursor)
aux.insertar_csv_a_tabla_postgres("input/test_cupid.csv", "test_cupid", cursor=cursor)


In [9]:
conn.commit()


# Parte 2

In [3]:
COLUMNAS = [
    "age",
    "height",
    "virgo",
    "taurus",
    "scorpio",
    "pisces",
    "libra",
    "leo",
    "gemini",
    "aries",
    "aquarius",
    "cancer",
    "sagittarius",
    "asian",
    "hispaniclatin",
    "black",
    "indian",
    "pacificislander",
    "nativeamerican",
    "middleeastern",
    "colorado",
    "newyork",
    "oregon",
    "arizona",
    "hawaii",
    "montana",
    "wisconsin",
    "virginia",
    "spain",
    "nevada",
    "illinois",
    "vietnam",
    "ireland",
    "louisiana",
    "michigan",
    "texas",
    "unitedkingdom",
    "massachusetts",
    "northcarolina",
    "idaho",
    "mississippi",
    "newjersey",
    "florida",
    "minnesota",
    "georgia",
    "utah",
    "washington",
    "westvirginia",
    "connecticut",
    "tennessee",
    "rhodeisland",
    "districtofcolumbia",
    "canada",
    "missouri",
    "germany",
    "pennsylvania",
    "netherlands",
    "switzerland",
    "mexico",
    "ohio",
    "agnosticism",
    "atheism",
    "catholicism",
    "buddhism",
    "judaism",
    "hinduism",
    "islam",
    "pro_dogs",
    "pro_cats",
    "spanish",
    "chinese",
    "french",
    "german",
    "single",
    "seeing_someone",
    "available",
    "employed",
    "income_between_25_50",
    "income_between_50_75",
    "income_over_75",
    "drugs_often",
    "drugs_sometimes",
    "drinks_notatall",
    "drinks_often",
    "drinks_rarely",
    "drinks_socially",
    "drinks_veryoften",
    "orientation_gay",
    "orientation_straight",
    "sex_m",
    "smokes_sometimes",
    "smokes_tryingtoquit",
    "smokes_whendrinking",
    "smokes_yes",
    "body_type_overweight",
    "body_type_regular",
    "education_high_school",
    "education_undergrad_university",
]

In [4]:
cursor.execute("SELECT * FROM train_cupid;")
train = pd.DataFrame(cursor.fetchall(), columns=COLUMNAS)


In [24]:
vectores_objetivos = ["single", "seeing_someone", "available"]
modelos = [
    GradientBoostingClassifier,
    AdaBoostClassifier,
    RandomForestClassifier,
    SVC,
    DecisionTreeClassifier,
    LogisticRegression,
    BernoulliNB,
]


In [25]:
modelos_entrenados = aux.entrenar_modelos_en_tanda(vectores_objetivos, modelos, train)


In [29]:
with open("modelos.pickle", "wb") as f:
    pickle.dump(modelos_entrenados, f)


# Parte 3

In [43]:
cursor.execute("SELECT * FROM test_cupid;")
test = pd.DataFrame(cursor.fetchall(), columns=COLUMNAS)

In [44]:
query_1 = test.query(
    "(atheism == 1) and (asian == 1) and (employed == 1) and (pro_dogs == 1.0) and (chinese == 1)"
)

query_2 = test.query(
    "(income_over_75 == 1) and (french == 1) and (german == 1) and (orientation_straight == 1) and (newyork == 1)"
)

query_3 = test.query(
    "(education_undergrad_university == 1) and (body_type_regular == 1) and (pro_dogs == 1.0) and (employed == 1)"
)

query_4 = test.query(
    "(taurus == 1) and (indian == 1) and (washington == 1) and (income_between_50_75 == 1) and (hinduism == 1)"
)

In [45]:
conjunto_queries = [query_1, query_2, query_3, query_4]
metricas = {"classification_report": classification_report, "roc_auc_score": roc_auc_score}

In [56]:
aux.testear_modelos_en_tanda(vectores_objetivos, modelos_entrenados, test, metricas)

Vector objetivo single - Modelo GradientBoostingClassifier()
--------------------------------------------
classification_report:               precision    recall  f1-score   support

           0       0.98      0.49      0.66      1616
           1       0.96      1.00      0.98     18327

    accuracy                           0.96     19943
   macro avg       0.97      0.75      0.82     19943
weighted avg       0.96      0.96      0.95     19943

roc_auc_score: 0.7461873023732231
--------------------------------------------

Vector objetivo single - Modelo AdaBoostClassifier()
--------------------------------------------
classification_report:               precision    recall  f1-score   support

           0       0.98      0.49      0.65      1616
           1       0.96      1.00      0.98     18327

    accuracy                           0.96     19943
   macro avg       0.97      0.74      0.82     19943
weighted avg       0.96      0.96      0.95     19943

roc_auc_score: 0

In [31]:
resultados_modelos


{'gradient_boost_pred_single': GradientBoostingClassifier(),
 'ada_boost_pred_single': AdaBoostClassifier(),
 'random_forest_pred_single': RandomForestClassifier(),
 'svc_pred_single': SVC(),
 'decision_tree_pred_single': DecisionTreeClassifier(),
 'logistic_pred_single': LogisticRegression(),
 'naive_bayes_pred_single': BernoulliNB(),
 'gradient_boost_pred_seeing_someone': GradientBoostingClassifier(),
 'ada_boost_pred_seeing_someone': AdaBoostClassifier(),
 'random_forest_pred_seeing_someone': RandomForestClassifier(),
 'svc_pred_seeing_someone': SVC(),
 'decision_tree_pred_seeing_someone': DecisionTreeClassifier(),
 'logistic_pred_seeing_someone': LogisticRegression(),
 'naive_bayes_pred_seeing_someone': BernoulliNB(),
 'gradient_boost_pred_available': GradientBoostingClassifier(),
 'ada_boost_pred_available': AdaBoostClassifier(),
 'random_forest_pred_available': RandomForestClassifier(),
 'svc_pred_available': SVC(),
 'decision_tree_pred_available': DecisionTreeClassifier(),
 'log